# # Third Molar Age Estimation (I3M) Exploration

In [ ]:
# ## 1. Imports & Utilities
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import albumentations as A
from scripts.tasks.age_estimation import compute_i3m, build_age_model

In [ ]:
# ## 2. Config
DATA_DIR = Path('../data/third_molar_age')
IMG_SIZE = (224,224)
BATCH_SIZE = 8

In [ ]:
# ## 3. Load Annotations & Compute I3M
ann = pd.read_csv(DATA_DIR/'annotations.csv')
ann['i3m'] = compute_i3m(ann)
ann['label'] = (ann['i3m'] < 0.08).astype(int)
ann.head()

In [ ]:
# ## 4. Visualize Bounding Boxes & I3M
examples = ann.sample(4)
for idx, row in examples.iterrows():
    img = plt.imread(DATA_DIR/'images'/row['filename'])
    x1,y1,x2,y2 = row[['crown_x1','crown_y1','crown_x2','crown_y2']]
    plt.figure(); plt.imshow(img, cmap='bone')
    plt.gca().add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1,
                                     edgecolor='r', fill=False))
    plt.title(f\"I3M={row['i3m']:.2f}, label={row['label']}\")
    plt.axis('off')

In [ ]:
# ## 5. Augmentation Pipeline Demo
aug = A.Compose([
    A.PadIfNeeded(min_height=300,min_width=300, border_mode=0),
    A.Rotate(limit=15)
])
sample = plt.imread(DATA_DIR/'images'/ann.loc[0,'filename'])
aug_img = aug(image=sample)['image']
plt.imshow(aug_img, cmap='bone'); plt.axis('off')

In [ ]:
# ## 6. Model Prototype & Quick Fit
model = build_age_model(IMG_SIZE)
model.summary()

In [ ]:
# ## Next Steps
# - Full dataset training via `train.py age_estimation …`
# - Compare to manual I3M thresholding